In [1]:
# !pip install openai==0.28

In [2]:
# ##uncomment this to use code in Google Colab
# !pip install fastchat
# !pip install wget
# !pip install evaluate
# !pip install sentence_transformers
# !pip install bert_score
# !pip install datasets

In [3]:
# ##uncomment this to use code in Google Colab
# from IPython.display import clear_output
# !git clone https://github.com/alfekka/lm-polygraph.git -b new_api
# %cd lm-polygraph
# %pip install .
# %cd src
# %pip install transformers rouge-score datasets openai
# !curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
# !apt-get install -y nodejs
# %cd lm_polygraph/app
# !npm install
# %cd ../../
# %cd /content/lm-polygraph/src
# %load_ext autoreload
# %autoreload 2

In [14]:
# ##uncomment this to use code in Google Colab
# %cd ..
# !pwd
# %cd /content/lm-polygraph/src

/content/lm-polygraph
/content/lm-polygraph
/content/lm-polygraph/src


In [15]:
# import nltk
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from lm_polygraph.generation_metrics.openai_fact_check import OpenAIFactCheck
from lm_polygraph.stat_calculators.extract_claims import Claim
import json
import pandas as pd
from tqdm import tqdm
russian_checker = OpenAIFactCheck('gpt-4o', language='ru')

In [17]:
from datasets import load_dataset
full = load_dataset("rvanova/person-bio-full")
full_claims = pd.DataFrame(full['test'])

In [18]:
full_claims.columns

Index(['question', 'bio', 'claim', 'sentence', 'tokens'], dtype='object')

In [19]:
for i, row in tqdm(full_claims.iterrows(), total=full_claims.shape[0]):
    claim = [Claim(claim_text=row['claim'], sentence=row['sentence'], aligned_tokens=row['tokens'])]
    check_dict = { "input_texts": [row['question']],
                  "claims": [claim]}
    chatgpt_response = russian_checker(check_dict, None, None )
    full_claims.at[i,'chatgpt_verdict'] = chatgpt_response[0]


100%|██████████| 298/298 [03:05<00:00,  1.61it/s]


In [21]:
full_claims.sample(frac=1).to_csv("../workdir/russian_fact_checked_claims_shuffled.csv", index=False)

full_claims.to_csv("../workdir/russian_fact_checked_claims.csv", index=False)